# Imports & setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path("..").resolve()))

from tqdm import trange
import torch
import matplotlib.pyplot as plt

from models.mlp import MLP
from data import get_dataloaders
from train_utils import train_epoch, evaluate


# Device & hyperparams

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 10
LR = 0.1


# Lancement des expériences

In [ ]:
train_loader, test_loader = get_dataloaders()

results = {}

for use_bn in [False, True]:
    name = "BN" if use_bn else "NoBN"
    model = MLP(use_bn=use_bn).to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=LR)

    results[name] = {
        "loss": [],
        "grad_norm": [],
        "train_acc": [],
        "test_acc": []
    }

    for epoch in trange(EPOCHS, desc=f"Epochs ({name})"):
        loss, gnorm = train_epoch(model, train_loader, optimizer, DEVICE)
        train_acc = evaluate(model, train_loader, DEVICE)
        test_acc = evaluate(model, test_loader, DEVICE)

        results[name]["loss"].append(loss)
        results[name]["grad_norm"].append(gnorm)
        results[name]["train_acc"].append(train_acc)
        results[name]["test_acc"].append(test_acc)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x3072 and 784x256)

# Plot loss

In [ ]:
epochs = range(1, EPOCHS + 1)

plt.plot(epochs, results["NoBN"]["loss"], label="No BN")
plt.plot(epochs, results["BN"]["loss"], label="With BN")
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss")
plt.legend()
plt.grid()
plt.show()


# Plot accuracy

In [ ]:
plt.plot(epochs, results["NoBN"]["test_acc"], label="Test No BN")
plt.plot(epochs, results["BN"]["test_acc"], label="Test BN")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Test Accuracy")
plt.legend()
plt.grid()
plt.show()


# Plot gradient norms

In [ ]:
plt.plot(epochs, results["NoBN"]["grad_norm"], label="No BN")
plt.plot(epochs, results["BN"]["grad_norm"], label="With BN")
plt.xlabel("Epoch")
plt.ylabel("Gradient Norm")
plt.title("Gradient Norm Evolution")
plt.legend()
plt.grid()
plt.show()
